In [ ]:
!pip install langchain-google-genai langchain-core langgraph pydantic --quiet

In [ ]:
import os
from typing import Annotated, TypedDict, Union, Literal
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, Field

In [ ]:
import os
import google.generativeai as genai
genai.configure(api_key=os.environ["EMAIL_API_KEY"])

for m in genai.list_models():
    print(m.name)

In [ ]:
!pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client

In [ ]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

Gmail authentication

In [ ]:
GMAIL_SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
def authorize_gmail():
    creds = None

    # Check if token already exists
    if os.path.isfile('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', GMAIL_SCOPES
        )

    # If no valid credentials, start manual OAuth
    if not creds or not creds.valid:
        oauth_flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json',
            scopes=GMAIL_SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob'
        )

        auth_link, _ = oauth_flow.authorization_url(
            access_type='offline',
            include_granted_scopes='true'
        )

        print("\n OPEN THIS LINK IN YOUR BROWSER:\n")
        print(auth_link)

        auth_code = input("\n Enter the authorization code here: ").strip()

        oauth_flow.fetch_token(code=auth_code)
        creds = oauth_flow.credentials

        # Save token
        with open('token.json', 'w') as f:
            f.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)


In [ ]:
gmail_service = authorize_gmail()
print(" Gmail authorization successful")


Testing access

In [ ]:
labels_response = gmail_service.users().labels().list(userId='me').execute()

print("\n Gmail Labels:")
for lbl in labels_response['labels']:
    print(lbl['name'])
